In [50]:
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from tqdm import tqdm

# 1. Scrapping non account (newest data)

In [196]:
output_file = pd.read_csv('22 November 2024.csv')
output_file = output_file.drop (['Imported Jobs'], axis=1 )
# output_file2 = output_file2.rename(columns={'Job status': 'Job type'})
output_file

,Company Name,Job Title,Location,Company Domain,Job LinkedIn URL,Posted On,Job type
0,Aurecon,"Consultant, Sustainability & Climate Change","Petaling Jaya, Selangor, Malaysia",aurecongroup.com,https://www.linkedin.com/jobs/view/consultant-...,2024-11-06T12:00:00.000Z,Full time
1,ERM,Corporate Sustainability & Climate Change Partner,"Bangkok, Bangkok City, Thailand",erm.com,https://www.linkedin.com/jobs/view/corporate-s...,2024-11-07T10:24:08.000Z,Full time
2,ERM,Consultant - Corporate Sustainability & Climat...,"Jakarta, Jakarta, Indonesia",erm.com,https://www.linkedin.com/jobs/view/consultant-...,2024-11-04T10:44:14.290Z,Full time
3,EY,"Manager, Climate Change and Sustainability Ser...","Singapore, Singapore",ey.com,https://www.linkedin.com/jobs/view/manager-cli...,2024-11-09T11:12:12.000Z,Full time
4,National University of Singapore,Senior Research Fellow (International Environm...,Singapore,nus.edu.sg,https://www.linkedin.com/jobs/view/senior-rese...,2024-11-11T10:40:21.000Z,Full time
...,...,...,...,...,...,...,...
129,TotalEnergies,Country Manager for Philippines (Solar C&I),"Taguig, National Capital Region, Philippines",totalenergies.com,https://www.linkedin.com/jobs/view/country-man...,2024-11-08T11:55:02.630Z,Contract
130,Aurecon,Environmental Officer (Pollution Control Offic...,"Pampanga, Central Luzon, Philippines",aurecongroup.com,https://www.linkedin.com/jobs/view/environment...,2024-11-11T12:00:00.000Z,Contract
131,Microsoft,Critical Environment Technician(CET) Apprentice,"North West Community Development Council, Sing...",microsoft.com,https://www.linkedin.com/jobs/view/critical-en...,2024-11-07T10:51:06.000Z,Internship
132,ZALORA Group,"Health, Safety & Environment (HSE) Intern - ba...","Shah Alam, Selangor, Malaysia",zalora.com,https://www.linkedin.com/jobs/view/health-safe...,2024-11-13T11:29:03.000Z,Internship


In [198]:
output_file['Posted On'] = pd.to_datetime(output_file['Posted On']).dt.date
output_file.rename(columns = {'Posted On': 'Format Date/Time'}, inplace = True)
# output_file = output_file.head(3)

In [182]:
# output_file = output_file.head(3)

In [200]:
output_file

,Company Name,Job Title,Location,Company Domain,Job LinkedIn URL,Format Date/Time,Job type
0,Aurecon,"Consultant, Sustainability & Climate Change","Petaling Jaya, Selangor, Malaysia",aurecongroup.com,https://www.linkedin.com/jobs/view/consultant-...,2024-11-06,Full time
1,ERM,Corporate Sustainability & Climate Change Partner,"Bangkok, Bangkok City, Thailand",erm.com,https://www.linkedin.com/jobs/view/corporate-s...,2024-11-07,Full time
2,ERM,Consultant - Corporate Sustainability & Climat...,"Jakarta, Jakarta, Indonesia",erm.com,https://www.linkedin.com/jobs/view/consultant-...,2024-11-04,Full time
3,EY,"Manager, Climate Change and Sustainability Ser...","Singapore, Singapore",ey.com,https://www.linkedin.com/jobs/view/manager-cli...,2024-11-09,Full time
4,National University of Singapore,Senior Research Fellow (International Environm...,Singapore,nus.edu.sg,https://www.linkedin.com/jobs/view/senior-rese...,2024-11-11,Full time
...,...,...,...,...,...,...,...
129,TotalEnergies,Country Manager for Philippines (Solar C&I),"Taguig, National Capital Region, Philippines",totalenergies.com,https://www.linkedin.com/jobs/view/country-man...,2024-11-08,Contract
130,Aurecon,Environmental Officer (Pollution Control Offic...,"Pampanga, Central Luzon, Philippines",aurecongroup.com,https://www.linkedin.com/jobs/view/environment...,2024-11-11,Contract
131,Microsoft,Critical Environment Technician(CET) Apprentice,"North West Community Development Council, Sing...",microsoft.com,https://www.linkedin.com/jobs/view/critical-en...,2024-11-07,Internship
132,ZALORA Group,"Health, Safety & Environment (HSE) Intern - ba...","Shah Alam, Selangor, Malaysia",zalora.com,https://www.linkedin.com/jobs/view/health-safe...,2024-11-13,Internship


In [202]:
linkedin_username  = "centerfoldloey71@gmail.com"
linkedin_password  = "170702!"

In [204]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

def init_driver():
    return webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

driver = init_driver()
driver.get("https://www.google.com")  # Open Google to test


def login_to_linkedin(driver, username, password):
    
    driver.get("https://www.linkedin.com/login")
    time.sleep(3)  

    username_input = driver.find_element(By.ID, "username")
    username_input.send_keys(username)

    password_input = driver.find_element(By.ID, "password")
    password_input.send_keys(password)

    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()

    time.sleep(5)  

login_to_linkedin(driver, linkedin_username, linkedin_password)

job_details_list = []

In [205]:
# def init_driver():
#     return webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

# # Start the driver
# driver = init_driver()

# List to store job details
job_details_list = []

# Iterate over rows in the DataFrame
for index, row in tqdm(output_file.iterrows(), total=output_file.shape[0]):

    # Extract job data from DataFrame
    URL = row['Job LinkedIn URL']
    company_name = row['Company Name']
    job_title = row['Job Title']
    location = row['Location']
    company_domain = row['Company Domain']
    format_date = row['Format Date/Time']
    job_type = row['Job type']

    try:
        # Open the job URL
        driver.get(URL)
        time.sleep(5)  # Wait for the page to load

        # Parse the page source
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Extract job descriptions
        job_desc_divs = soup.find_all(
            "div",
            {"class": "jobs-box__html-content jobs-description-content__text t-14 t-normal jobs-description-content__text--stretch"}
        )
        job_descriptions = ''.join([div.decode_contents().strip().replace('\n', '') for div in job_desc_divs])

        # Extract company image URLs
        img_tags = soup.find_all(
            'img',
            class_='ivm-view-attr__img--centered EntityPhoto-square-2 evi-image lazy-image ember-view'
        )
        image_urls = [img['src'] for img in img_tags]

        # Extract job status
        job_status_div = soup.find_all('span', class_='artdeco-inline-feedback__message')
        job_status_update = [div.get_text(strip=True) for div in job_status_div]

        # Extract job type information
        job_type_span = soup.find_all(
            'li',
            class_='job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight'
        )
        job_cat_final = [
            span.get_text(strip=True) for span in job_type_span
            if 'Gabungan' in span.get_text() or 'Di Kantor' in span.get_text() or 'Jarak Jauh' in span.get_text()
        ]

        # Skip jobs marked as 'Expired'
        if any('Expired' in status for status in job_status_update):
            continue

    except Exception as e:
        print(f"Failed to load {URL}: {e}")

        # Restart the driver on failure
        driver.quit()
        driver = init_driver()

        # Placeholder values in case of an error
        job_status_update = ["Error loading job status"]
        job_cat_final = []
        job_descriptions = ""
        image_urls = []

    # Append job details to the list
    job_details_list.append({
        'Job LinkedIn URL': URL,
        'Company Name': company_name,
        'Job Title': job_title,
        'Location': location,
        'Company Domain': company_domain,
        'Format Date/Time': format_date,
        'Job type': job_type,
        'Update Job Status': job_status_update,
        'Job Descriptions': job_descriptions,
        'Company Image URLs': image_urls,   
        'Job category': job_cat_final
    })


output_file_temp = pd.DataFrame(job_details_list)
output_file_temp

100%|██████████| 134/134 [14:53<00:00,  6.67s/it]


,Job LinkedIn URL,Company Name,Job Title,Location,Company Domain,Format Date/Time,Job type,Update Job Status,Job Descriptions,Company Image URLs,Job category
0,https://www.linkedin.com/jobs/view/consultant-...,Aurecon,"Consultant, Sustainability & Climate Change","Petaling Jaya, Selangor, Malaysia",aurecongroup.com,2024-11-06,Full time,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/D560BAQH...,[GabunganPenuh waktuSenior tingkat menengah]
1,https://www.linkedin.com/jobs/view/corporate-s...,ERM,Corporate Sustainability & Climate Change Partner,"Bangkok, Bangkok City, Thailand",erm.com,2024-11-07,Full time,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/D4D0BAQF...,[Di KantorPenuh waktuSenior tingkat menengah]
2,https://www.linkedin.com/jobs/view/consultant-...,ERM,Consultant - Corporate Sustainability & Climat...,"Jakarta, Jakarta, Indonesia",erm.com,2024-11-04,Full time,[Tidak lagi menerima lamaran],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/D4D0BAQF...,[Di KantorPenuh waktuSenior tingkat menengah]
3,https://www.linkedin.com/jobs/view/manager-cli...,EY,"Manager, Climate Change and Sustainability Ser...","Singapore, Singapore",ey.com,2024-11-09,Full time,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/C510BAQG...,[Di KantorPenuh waktuSenior tingkat menengah]
4,https://www.linkedin.com/jobs/view/senior-rese...,National University of Singapore,Senior Research Fellow (International Environm...,Singapore,nus.edu.sg,2024-11-11,Full time,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/C4D0BAQG...,[Di KantorPenuh waktu]
...,...,...,...,...,...,...,...,...,...,...,...
129,https://www.linkedin.com/jobs/view/country-man...,TotalEnergies,Country Manager for Philippines (Solar C&I),"Taguig, National Capital Region, Philippines",totalenergies.com,2024-11-08,Contract,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/C4D0BAQG...,[Di KantorKontrakDirektur]
130,https://www.linkedin.com/jobs/view/environment...,Aurecon,Environmental Officer (Pollution Control Offic...,"Pampanga, Central Luzon, Philippines",aurecongroup.com,2024-11-11,Contract,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/D560BAQH...,[Di KantorKontrak]
131,https://www.linkedin.com/jobs/view/critical-en...,Microsoft,Critical Environment Technician(CET) Apprentice,"North West Community Development Council, Sing...",microsoft.com,2024-11-07,Internship,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/C560BAQE...,[Di KantorMagang]
132,https://www.linkedin.com/jobs/view/health-safe...,ZALORA Group,"Health, Safety & Environment (HSE) Intern - ba...","Shah Alam, Selangor, Malaysia",zalora.com,2024-11-13,Internship,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/C560BAQF...,[Di KantorMagangMagang]


In [208]:
output_file_temp

,Job LinkedIn URL,Company Name,Job Title,Location,Company Domain,Format Date/Time,Job type,Update Job Status,Job Descriptions,Company Image URLs,Job category
0,https://www.linkedin.com/jobs/view/consultant-...,Aurecon,"Consultant, Sustainability & Climate Change","Petaling Jaya, Selangor, Malaysia",aurecongroup.com,2024-11-06,Full time,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/D560BAQH...,[GabunganPenuh waktuSenior tingkat menengah]
1,https://www.linkedin.com/jobs/view/corporate-s...,ERM,Corporate Sustainability & Climate Change Partner,"Bangkok, Bangkok City, Thailand",erm.com,2024-11-07,Full time,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/D4D0BAQF...,[Di KantorPenuh waktuSenior tingkat menengah]
2,https://www.linkedin.com/jobs/view/consultant-...,ERM,Consultant - Corporate Sustainability & Climat...,"Jakarta, Jakarta, Indonesia",erm.com,2024-11-04,Full time,[Tidak lagi menerima lamaran],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/D4D0BAQF...,[Di KantorPenuh waktuSenior tingkat menengah]
3,https://www.linkedin.com/jobs/view/manager-cli...,EY,"Manager, Climate Change and Sustainability Ser...","Singapore, Singapore",ey.com,2024-11-09,Full time,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/C510BAQG...,[Di KantorPenuh waktuSenior tingkat menengah]
4,https://www.linkedin.com/jobs/view/senior-rese...,National University of Singapore,Senior Research Fellow (International Environm...,Singapore,nus.edu.sg,2024-11-11,Full time,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/C4D0BAQG...,[Di KantorPenuh waktu]
...,...,...,...,...,...,...,...,...,...,...,...
129,https://www.linkedin.com/jobs/view/country-man...,TotalEnergies,Country Manager for Philippines (Solar C&I),"Taguig, National Capital Region, Philippines",totalenergies.com,2024-11-08,Contract,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/C4D0BAQG...,[Di KantorKontrakDirektur]
130,https://www.linkedin.com/jobs/view/environment...,Aurecon,Environmental Officer (Pollution Control Offic...,"Pampanga, Central Luzon, Philippines",aurecongroup.com,2024-11-11,Contract,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/D560BAQH...,[Di KantorKontrak]
131,https://www.linkedin.com/jobs/view/critical-en...,Microsoft,Critical Environment Technician(CET) Apprentice,"North West Community Development Council, Sing...",microsoft.com,2024-11-07,Internship,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/C560BAQE...,[Di KantorMagang]
132,https://www.linkedin.com/jobs/view/health-safe...,ZALORA Group,"Health, Safety & Environment (HSE) Intern - ba...","Shah Alam, Selangor, Malaysia",zalora.com,2024-11-13,Internship,[],"<h2 class=""text-heading-large""> T...",[https://media.licdn.com/dms/image/v2/C560BAQF...,[Di KantorMagangMagang]


In [210]:
# saving data terbaru jadi ke csv
output_file_temp.to_csv('Green jobs - Data - 22 November.csv', encoding='utf-8-sig')

## Untuk nambahin remote/hybrid/on site

In [220]:
output_file_temp = pd.read_csv('Green jobs - Data - 22 November.csv')
output_file_temp

,Unnamed: 0,Job LinkedIn URL,Company Name,Job Title,Location,Company Domain,Format Date/Time,Job type,Update Job Status,Job Descriptions,Company Image URLs,Job category
0,0,https://www.linkedin.com/jobs/view/consultant-...,Aurecon,"Consultant, Sustainability & Climate Change","Petaling Jaya, Selangor, Malaysia",aurecongroup.com,2024-11-06,Full time,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/D560BAQ...,['GabunganPenuh waktuSenior tingkat menengah']
1,1,https://www.linkedin.com/jobs/view/corporate-s...,ERM,Corporate Sustainability & Climate Change Partner,"Bangkok, Bangkok City, Thailand",erm.com,2024-11-07,Full time,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/D4D0BAQ...,['Di KantorPenuh waktuSenior tingkat menengah']
2,2,https://www.linkedin.com/jobs/view/consultant-...,ERM,Consultant - Corporate Sustainability & Climat...,"Jakarta, Jakarta, Indonesia",erm.com,2024-11-04,Full time,['Tidak lagi menerima lamaran'],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/D4D0BAQ...,['Di KantorPenuh waktuSenior tingkat menengah']
3,3,https://www.linkedin.com/jobs/view/manager-cli...,EY,"Manager, Climate Change and Sustainability Ser...","Singapore, Singapore",ey.com,2024-11-09,Full time,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/C510BAQ...,['Di KantorPenuh waktuSenior tingkat menengah']
4,4,https://www.linkedin.com/jobs/view/senior-rese...,National University of Singapore,Senior Research Fellow (International Environm...,Singapore,nus.edu.sg,2024-11-11,Full time,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/C4D0BAQ...,['Di KantorPenuh waktu']
...,...,...,...,...,...,...,...,...,...,...,...,...
129,129,https://www.linkedin.com/jobs/view/country-man...,TotalEnergies,Country Manager for Philippines (Solar C&I),"Taguig, National Capital Region, Philippines",totalenergies.com,2024-11-08,Contract,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/C4D0BAQ...,['Di KantorKontrakDirektur']
130,130,https://www.linkedin.com/jobs/view/environment...,Aurecon,Environmental Officer (Pollution Control Offic...,"Pampanga, Central Luzon, Philippines",aurecongroup.com,2024-11-11,Contract,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/D560BAQ...,['Di KantorKontrak']
131,131,https://www.linkedin.com/jobs/view/critical-en...,Microsoft,Critical Environment Technician(CET) Apprentice,"North West Community Development Council, Sing...",microsoft.com,2024-11-07,Internship,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/C560BAQ...,['Di KantorMagang']
132,132,https://www.linkedin.com/jobs/view/health-safe...,ZALORA Group,"Health, Safety & Environment (HSE) Intern - ba...","Shah Alam, Selangor, Malaysia",zalora.com,2024-11-13,Internship,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/C560BAQ...,['Di KantorMagangMagang']


In [222]:
category = output_file_temp['Job type']

In [224]:
def clean_job_category(cat):
    if not cat or len(cat) == 0:
        return ''  
    
    try:
        cat_list = eval(cat)
        if not isinstance(cat_list, list) or len(cat_list) == 0:
            return 'Unknown'
        
        cat_str = cat_list[0]  

        if 'Di Kantor' in cat_str and 'Magang' in cat_str:
            return 'Internship, 📌On-site'
        elif 'Jarak Jauh' in cat_str and 'Magang' in cat_str:
            return 'Internship, 📌Remote'
        elif 'Gabungan' in cat_str and 'Magang' in cat_str:
            return 'Internship, 📌Hybrid'
        elif 'Jarak Jauh' in cat_str and 'Kontrak' in cat_str:
            return 'Contract, 📌Remote'
        elif 'Gabungan' in cat_str and 'Kontrak' in cat_str:
            return 'Contract, 📌Hybrid'
        elif 'Di Kantor' in cat_str and 'Kontrak' in cat_str:
            return 'Contract, 📌On-site'
        elif 'Jarak Jauh' in cat_str and 'Penuh waktu' in cat_str:
            return 'Full time, 📌Remote'
        elif 'Gabungan' in cat_str and 'Penuh waktu' in cat_str:
            return 'Full time, 📌Hybrid'
        elif 'Di Kantor' in cat_str and 'Penuh waktu' in cat_str:
            return 'Full time, 📌On-site'
        else:
            return 'Unknown'  

    except (SyntaxError, ValueError, TypeError) as e:
        print(f"Error processing {cat}: {e}")
        return 'Unknown'

output_file_temp['Cleaned Job category'] = output_file_temp['Job category'].apply(clean_job_category)

In [226]:
output_file_temp

,Unnamed: 0,Job LinkedIn URL,Company Name,Job Title,Location,Company Domain,Format Date/Time,Job type,Update Job Status,Job Descriptions,Company Image URLs,Job category,Cleaned Job category
0,0,https://www.linkedin.com/jobs/view/consultant-...,Aurecon,"Consultant, Sustainability & Climate Change","Petaling Jaya, Selangor, Malaysia",aurecongroup.com,2024-11-06,Full time,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/D560BAQ...,['GabunganPenuh waktuSenior tingkat menengah'],"Full time, 📌Hybrid"
1,1,https://www.linkedin.com/jobs/view/corporate-s...,ERM,Corporate Sustainability & Climate Change Partner,"Bangkok, Bangkok City, Thailand",erm.com,2024-11-07,Full time,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/D4D0BAQ...,['Di KantorPenuh waktuSenior tingkat menengah'],"Full time, 📌On-site"
2,2,https://www.linkedin.com/jobs/view/consultant-...,ERM,Consultant - Corporate Sustainability & Climat...,"Jakarta, Jakarta, Indonesia",erm.com,2024-11-04,Full time,['Tidak lagi menerima lamaran'],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/D4D0BAQ...,['Di KantorPenuh waktuSenior tingkat menengah'],"Full time, 📌On-site"
3,3,https://www.linkedin.com/jobs/view/manager-cli...,EY,"Manager, Climate Change and Sustainability Ser...","Singapore, Singapore",ey.com,2024-11-09,Full time,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/C510BAQ...,['Di KantorPenuh waktuSenior tingkat menengah'],"Full time, 📌On-site"
4,4,https://www.linkedin.com/jobs/view/senior-rese...,National University of Singapore,Senior Research Fellow (International Environm...,Singapore,nus.edu.sg,2024-11-11,Full time,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/C4D0BAQ...,['Di KantorPenuh waktu'],"Full time, 📌On-site"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,129,https://www.linkedin.com/jobs/view/country-man...,TotalEnergies,Country Manager for Philippines (Solar C&I),"Taguig, National Capital Region, Philippines",totalenergies.com,2024-11-08,Contract,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/C4D0BAQ...,['Di KantorKontrakDirektur'],"Contract, 📌On-site"
130,130,https://www.linkedin.com/jobs/view/environment...,Aurecon,Environmental Officer (Pollution Control Offic...,"Pampanga, Central Luzon, Philippines",aurecongroup.com,2024-11-11,Contract,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/D560BAQ...,['Di KantorKontrak'],"Contract, 📌On-site"
131,131,https://www.linkedin.com/jobs/view/critical-en...,Microsoft,Critical Environment Technician(CET) Apprentice,"North West Community Development Council, Sing...",microsoft.com,2024-11-07,Internship,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/C560BAQ...,['Di KantorMagang'],"Internship, 📌On-site"
132,132,https://www.linkedin.com/jobs/view/health-safe...,ZALORA Group,"Health, Safety & Environment (HSE) Intern - ba...","Shah Alam, Selangor, Malaysia",zalora.com,2024-11-13,Internship,[],"<h2 class=""text-heading-large""> T...",['https://media.licdn.com/dms/image/v2/C560BAQ...,['Di KantorMagangMagang'],"Internship, 📌On-site"


In [228]:
output_file_temp.to_csv('Green jobs - Data - 22 November - Fix.csv', encoding='utf-8-sig')

In [129]:
new_21_domain = pd.read_csv('updated_21_with_domains.csv')
new_21_domain

,Job LinkedIn URL,Company Name,Job Title,Location,Company Domain,Format Date/Time,Job type,Job Descriptions,Company Image URLs,Ai Summarized,Keyword,Update Job Status
0,https://www.linkedin.com/jobs/view/procurement...,AB InBev Southeast Asia,Procurement & Sustainability Associate Director,"Ho Chi Minh City, Vietnam",ab-inbev.com,2024-10-08,"Full time, 📌 On-site",<p><strong>Position Title: Procurement &amp; S...,https://media.licdn.com/dms/image/v2/C4D0BAQGS...,<p><strong>Position Title: Procurement &amp; S...,Sustainability,[]
1,https://www.linkedin.com/jobs/view/environment...,Abt Global,Environmental Consultant,Thailand,abtassoc.com,2024-10-01,"Part time, 📌 Remote",<strong>Job Description<br/><br/></strong><str...,https://media.licdn.com/dms/image/v2/D4E0BAQHl...,<strong>Job Description<br/><br/></strong><str...,Environment,['Tidak lagi menerima lamaran']
2,https://www.linkedin.com/jobs/view/topic-advis...,Accenture in the Philippines,Topic Advisory Sustainability,"Taguig, National Capital Region, Philippines",accenture.com,2024-10-07,"Full time, 📌 On-site",<strong>Topic Advisory Senior Manager (L6) Sus...,https://media.licdn.com/dms/image/v2/D560BAQEP...,<strong>Topic Advisory Senior Manager (L6) Sus...,Sustainability,[]
3,https://www.linkedin.com/jobs/view/sustainabil...,Adecco,Sustainability Manager [Built Environment MNC],"Singapore, Singapore",adecco.com,2024-10-08,"Full time, 📌 On-site",Adecco is collaborating with a Global Built En...,https://media.licdn.com/dms/image/v2/C510BAQEe...,Adecco is collaborating with a Global Built En...,"Sustainability, Environment",['Tidak lagi menerima lamaran']
4,https://www.linkedin.com/jobs/view/senior-cons...,AFRY,Senior Consultant / Consultant - Renewable Energy,"Singapore, Singapore",afry.com,2024-10-05,Full time,<strong>Company Description<br/><br/></strong>...,https://media.licdn.com/dms/image/v2/C4E0BAQEA...,<strong>Company Description<br/><br/></strong>...,"Renewable Energy, Energy",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
290,https://www.linkedin.com/jobs/view/human-resou...,Veolia Water Technologies & Solutions,Human Resources Specialist (1 Year Renewable C...,"Singapore, Singapore",watertechnologies.com,2024-09-16,"Contract, 📌 Hybrid",<p><strong><u>Primary Responsibilities:</u></s...,https://media.licdn.com/dms/image/v2/D560BAQFk...,<p><strong>Job Description</strong></p>\n<ul>\...,Sustainability,['Error loading job status']
291,https://www.linkedin.com/jobs/view/marketing-i...,Oil and Gas Job Search Ltd,Marketing Intern (Sustainability),"Singapore, Singapore",oilandgasjobsearch.com,2024-09-12,Internship,Do you dream of working in a company that is d...,https://media.licdn.com/dms/image/v2/C4D0BAQEq...,<p><strong>Company Description</strong></p>\n<...,Sustainability,['Error loading job status']
292,https://www.linkedin.com/jobs/view/intern-sust...,Marriott International,"Intern, Sustainability (Jan Jun 2025)","Singapore, Singapore",marriott.com,2024-09-12,Internship,<strong>Job Number</strong> 24146951<br/><br/>...,https://media.licdn.com/dms/image/v2/C4E0BAQGY...,<p><strong>Job Description</strong></p>\n<p>Th...,Sustainability,['Error loading job status']
293,https://www.linkedin.com/jobs/view/factory-sus...,Seagate Technology,Factory Sustainable Engineering Intern,"Samut Prakan, Samut Prakan, Thailand",seagate.com,2024-09-11,"Internship, 📌 On-site",<strong>About Our Group<br/><br/></strong>Fact...,https://media.licdn.com/dms/image/v2/D560BAQE_...,<p><strong>Job Description</strong></p>\n<p>Fa...,Sustainability,['Error loading job status']
